In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)

warnings.filterwarnings("ignore")

In [ ]:
pre = "MH105"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
import sccoda
from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz
import sccoda.datasets as scd

### Human

In [ ]:
def make_df_annotation_azimut1_5_scanvi_v2(df):
    df['cluster_azimut1_5_scanvi_v2'] = df['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})
    return df

In [ ]:
path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)

In [ ]:
species = 'human'
_,_,_,base_anndata_objects_H24 = h.return_local_paths(drive,pre='H24',add_path = False)
adata_human = sc.read_h5ad(os.path.join(base_anndata_objects_H24,'H24' + '_' + species + '_anno_celltypes_v0.h5ad'))
adata_human.obs = pd.merge(adata_human.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_human.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_human.obs)
adata_human.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)

Human

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    celltype_col = 'cluster_azimut1_5_scanvi_v2'
    marker_size_umap = 60
    
    adata_human_ct = adata_human[adata_human.obs[celltype_col] == celltype]
    adata_human_ct.obs['scvi_clusters'] = adata_human_ct.obs['scvi_clusters'].astype('category')
    
    h.prepare_umap(adata_human_ct)
    
    x_min = np.min(adata_human_ct.obsm['X_umap'][:,0])
    x_max = np.max(adata_human_ct.obsm['X_umap'][:,0])
    y_min = np.min(adata_human_ct.obsm['X_umap'][:,1])
    y_max = np.max(adata_human_ct.obsm['X_umap'][:,1])
    
    adata_ct_00hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '00hr']
    adata_ct_06hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '06hr']
    adata_ct_24hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '24hr']
    
    cmap_style = 'tab20'
    adata_ct_00hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_00hr.obs['scvi_clusters'].nunique()]
    adata_ct_06hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_06hr.obs['scvi_clusters'].nunique()]
    adata_ct_24hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_24hr.obs['scvi_clusters'].nunique()]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    sc.pl.umap(adata_ct_00hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[0])
    axes[0].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[0].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[0].set_title(celltype + " - 00hr")
    
    sc.pl.umap(adata_ct_06hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[1])
    axes[1].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[1].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[1].set_title(celltype + " - 06hr")
    
    sc.pl.umap(adata_ct_24hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[2])
    axes[2].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[2].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[2].set_title(celltype + " - 24hr")
    
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + 'umaps_scvi_clusters.pdf'),bbox_inches='tight')
    plt.show()

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    adata_human_ct = adata_human[adata_human.obs[celltype_col] == celltype]
    adata_human_ct.obs['scvi_clusters'] = adata_human_ct.obs['scvi_clusters'].astype('category')
    
    df_human_ct =  adata_human_ct.obs[[celltype_col,'timepoint','scvi_clusters']]
    
    cell_counts_human = df_human_ct.groupby(['timepoint','scvi_clusters']).size().reset_index(name='count')
    
    cell_counts_pivot_human = cell_counts_human.pivot(index='timepoint',columns = 'scvi_clusters',values = 'count').reset_index()
    
    scoda_obj_human = dat.from_scanpy(
        adata_human_ct,
        cell_type_identifier='scvi_clusters',
        sample_identifier="timepoint",
        )
    
    #save_path_human_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_human.h5ad')
    #scoda_obj_human.write_h5ad(save_path_human_h5ad)
    
    df_all_human = pd.DataFrame(scoda_obj_human.X,columns = scoda_obj_human.var.index,index = scoda_obj_human.obs.index)
    proportions = df_all_human.div(df_all_human.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'))
    
    color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
    proportions.plot(kind='bar', figsize=(8, 6), width=0.9, color=color_timepoints)
    plt.title('Cell type proportions - Human - ' + str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0,1.02])
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + '_composition_barplot_human.pdf'),bbox_inches = 'tight')
    plt.show()

    scoda_obj_human.obs['Condition'] = scoda_obj_human.obs.index 
    scoda_model = mod.CompositionalAnalysis(scoda_obj_human, formula="Condition", reference_cell_type="automatic")
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    sim_results.save(save_path)

In [ ]:
for celltype in ['NK+Proliferating']:
    celltype_col = 'cluster_azimut1_5_scanvi_v2'
    marker_size_umap = 60
    
    adata_human_ct = adata_human[adata_human.obs[celltype_col] == celltype]
    adata_human_ct.obs['cluster_azimut1_5_scanvi'] = adata_human_ct.obs['cluster_azimut1_5_scanvi'].astype('category')
    
    h.prepare_umap(adata_human_ct)
    
    x_min = np.min(adata_human_ct.obsm['X_umap'][:,0])
    x_max = np.max(adata_human_ct.obsm['X_umap'][:,0])
    y_min = np.min(adata_human_ct.obsm['X_umap'][:,1])
    y_max = np.max(adata_human_ct.obsm['X_umap'][:,1])
    
    adata_ct_00hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '00hr']
    adata_ct_06hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '06hr']
    adata_ct_24hr = adata_human_ct[adata_human_ct.obs['timepoint'] == '24hr']
    
    cmap_style = 'tab20'
    adata_ct_00hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_00hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    adata_ct_06hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_06hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    adata_ct_24hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_24hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    sc.pl.umap(adata_ct_00hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[0])
    axes[0].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[0].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[0].set_title(celltype + " - 00hr")
    
    sc.pl.umap(adata_ct_06hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[1])
    axes[1].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[1].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[1].set_title(celltype + " - 06hr")
    
    sc.pl.umap(adata_ct_24hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[2])
    axes[2].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[2].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[2].set_title(celltype + " - 24hr")
    
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + 'umaps_cluster_azimut1_5_scanvi_human.pdf'),bbox_inches='tight')
    plt.show()

In [ ]:
for celltype in ['NK+Proliferating']:
    adata_human_ct = adata_human[adata_human.obs[celltype_col] == celltype]
    adata_human_ct.obs['cluster_azimut1_5_scanvi'] = adata_human_ct.obs['cluster_azimut1_5_scanvi'].astype('category')
    
    df_human_ct =  adata_human_ct.obs[[celltype_col,'timepoint','cluster_azimut1_5_scanvi']]
    
    cell_counts_human = df_human_ct.groupby(['timepoint','cluster_azimut1_5_scanvi']).size().reset_index(name='count')
    
    cell_counts_pivot_human = cell_counts_human.pivot(index='timepoint',columns = 'cluster_azimut1_5_scanvi',values = 'count').reset_index()
    
    scoda_obj_human = dat.from_scanpy(
        adata_human_ct,
        cell_type_identifier='cluster_azimut1_5_scanvi',
        sample_identifier="timepoint",
        )
    
    #save_path_human_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_human.h5ad')
    #scoda_obj_human.write_h5ad(save_path_human_h5ad)
    
    df_all_human = pd.DataFrame(scoda_obj_human.X,columns = scoda_obj_human.var.index,index = scoda_obj_human.obs.index)
    proportions = df_all_human.div(df_all_human.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_human.csv'))
    
    color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
    proportions.plot(kind='bar', figsize=(8, 6), width=0.9, color=color_timepoints)
    plt.title('Cell type proportions - human - ' + str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0,1.02])
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + '_composition_barplot_human_cluster_azimut1_5_scanvi.pdf'),bbox_inches = 'tight')
    plt.show()

    scoda_obj_human.obs['Condition'] = scoda_obj_human.obs.index 
    scoda_model = mod.CompositionalAnalysis(scoda_obj_human, formula="Condition", reference_cell_type="automatic")
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_human.pkl')
    sim_results.save(save_path)

### Cyno

In [ ]:
species = 'cyno'
_,_,_,base_anndata_objects_M24 = h.return_local_paths(drive,pre='M24',add_path = False)
adata_cyno = sc.read_h5ad(os.path.join(base_anndata_objects_M24,'M24' + '_' + species + '_anno_celltypes_v0.h5ad'))
adata_cyno.obs = pd.merge(adata_cyno.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_cyno.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_cyno.obs)
adata_cyno.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    celltype_col = 'cluster_azimut1_5_scanvi_v2'
    marker_size_umap = 60
    
    adata_cyno_ct = adata_cyno[adata_cyno.obs[celltype_col] == celltype]
    adata_cyno_ct.obs['scvi_clusters'] = adata_cyno_ct.obs['scvi_clusters'].astype('category')
    
    h.prepare_umap(adata_cyno_ct)
    
    x_min = np.min(adata_cyno_ct.obsm['X_umap'][:,0])
    x_max = np.max(adata_cyno_ct.obsm['X_umap'][:,0])
    y_min = np.min(adata_cyno_ct.obsm['X_umap'][:,1])
    y_max = np.max(adata_cyno_ct.obsm['X_umap'][:,1])
    
    adata_ct_00hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '00hr']
    adata_ct_06hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '06hr']
    adata_ct_24hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '24hr']
    
    cmap_style = 'tab20'
    adata_ct_00hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_00hr.obs['scvi_clusters'].nunique()]
    adata_ct_06hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_06hr.obs['scvi_clusters'].nunique()]
    adata_ct_24hr.uns['scvi_clusters_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_24hr.obs['scvi_clusters'].nunique()]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    sc.pl.umap(adata_ct_00hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[0])
    axes[0].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[0].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[0].set_title(celltype + " - 00hr")
    
    sc.pl.umap(adata_ct_06hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[1])
    axes[1].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[1].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[1].set_title(celltype + " - 06hr")
    
    sc.pl.umap(adata_ct_24hr, color='scvi_clusters', show=False, size=marker_size_umap, ax=axes[2])
    axes[2].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[2].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[2].set_title(celltype + " - 24hr")
    
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + 'umaps_scvi_clusters_cyno.pdf'),bbox_inches='tight')
    plt.show()

In [ ]:
for celltype in ['NK+Proliferating']:
    celltype_col = 'cluster_azimut1_5_scanvi_v2'
    marker_size_umap = 60
    
    adata_cyno_ct = adata_cyno[adata_cyno.obs[celltype_col] == celltype]
    adata_cyno_ct.obs['cluster_azimut1_5_scanvi'] = adata_cyno_ct.obs['cluster_azimut1_5_scanvi'].astype('category')
    
    h.prepare_umap(adata_cyno_ct)
    
    x_min = np.min(adata_cyno_ct.obsm['X_umap'][:,0])
    x_max = np.max(adata_cyno_ct.obsm['X_umap'][:,0])
    y_min = np.min(adata_cyno_ct.obsm['X_umap'][:,1])
    y_max = np.max(adata_cyno_ct.obsm['X_umap'][:,1])
    
    adata_ct_00hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '00hr']
    adata_ct_06hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '06hr']
    adata_ct_24hr = adata_cyno_ct[adata_cyno_ct.obs['timepoint'] == '24hr']
    
    cmap_style = 'tab20'
    adata_ct_00hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_00hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    adata_ct_06hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_06hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    adata_ct_24hr.uns['cluster_azimut1_5_scanvi_colors'] = plt.get_cmap(cmap_style).colors[:adata_ct_24hr.obs['cluster_azimut1_5_scanvi'].nunique()]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    sc.pl.umap(adata_ct_00hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[0])
    axes[0].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[0].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[0].set_title(celltype + " - 00hr")
    
    sc.pl.umap(adata_ct_06hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[1])
    axes[1].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[1].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[1].set_title(celltype + " - 06hr")
    
    sc.pl.umap(adata_ct_24hr, color='cluster_azimut1_5_scanvi', show=False, size=marker_size_umap, ax=axes[2])
    axes[2].set_xlim(np.floor(x_min-1), np.floor(x_max + 2))
    axes[2].set_ylim(np.floor(y_min-1), np.floor(y_max + 2))
    axes[2].set_title(celltype + " - 24hr")
    
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + 'umaps_cluster_azimut1_5_scanvi_cyno.pdf'),bbox_inches='tight')
    plt.show()

In [ ]:
for celltype in ['CD8 T','CD16 Mono','NK+Proliferating']:
    adata_cyno_ct = adata_cyno[adata_cyno.obs[celltype_col] == celltype]
    adata_cyno_ct.obs['scvi_clusters'] = adata_cyno_ct.obs['scvi_clusters'].astype('category')
    
    df_cyno_ct =  adata_cyno_ct.obs[[celltype_col,'timepoint','scvi_clusters']]
    
    cell_counts_cyno = df_cyno_ct.groupby(['timepoint','scvi_clusters']).size().reset_index(name='count')
    
    cell_counts_pivot_cyno = cell_counts_cyno.pivot(index='timepoint',columns = 'scvi_clusters',values = 'count').reset_index()
    
    scoda_obj_cyno = dat.from_scanpy(
        adata_cyno_ct,
        cell_type_identifier='scvi_clusters',
        sample_identifier="timepoint",
        )
    
    #save_path_cyno_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_cyno.h5ad')
    #scoda_obj_cyno.write_h5ad(save_path_cyno_h5ad)
    
    df_all_cyno = pd.DataFrame(scoda_obj_cyno.X,columns = scoda_obj_cyno.var.index,index = scoda_obj_cyno.obs.index)
    proportions = df_all_cyno.div(df_all_cyno.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_cyno.csv'))
    
    color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
    proportions.plot(kind='bar', figsize=(8, 6), width=0.9, color=color_timepoints)
    plt.title('Cell type proportions - cyno - ' + str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0,1.02])
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + '_composition_barplot_cyno.pdf'),bbox_inches = 'tight')
    plt.show()

    scoda_obj_cyno.obs['Condition'] = scoda_obj_cyno.obs.index 
    scoda_model = mod.CompositionalAnalysis(scoda_obj_cyno, formula="Condition", reference_cell_type="automatic")
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_cyno.pkl')
    sim_results.save(save_path)

In [ ]:
for celltype in ['NK+Proliferating']:
    adata_cyno_ct = adata_cyno[adata_cyno.obs[celltype_col] == celltype]
    adata_cyno_ct.obs['cluster_azimut1_5_scanvi'] = adata_cyno_ct.obs['cluster_azimut1_5_scanvi'].astype('category')
    
    df_cyno_ct =  adata_cyno_ct.obs[[celltype_col,'timepoint','cluster_azimut1_5_scanvi']]
    
    cell_counts_cyno = df_cyno_ct.groupby(['timepoint','cluster_azimut1_5_scanvi']).size().reset_index(name='count')
    
    cell_counts_pivot_cyno = cell_counts_cyno.pivot(index='timepoint',columns = 'cluster_azimut1_5_scanvi',values = 'count').reset_index()
    
    scoda_obj_cyno = dat.from_scanpy(
        adata_cyno_ct,
        cell_type_identifier='cluster_azimut1_5_scanvi',
        sample_identifier="timepoint",
        )
    
    #save_path_cyno_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_cyno.h5ad')
    #scoda_obj_cyno.write_h5ad(save_path_cyno_h5ad)
    
    df_all_cyno = pd.DataFrame(scoda_obj_cyno.X,columns = scoda_obj_cyno.var.index,index = scoda_obj_cyno.obs.index)
    proportions = df_all_cyno.div(df_all_cyno.sum(axis=0), axis=1)
    proportions = proportions.T
    
    proportions.to_csv(os.path.join(base_table_path,pre + '_' + str(celltype) +  '_proportions_cyno.csv'))
    
    color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
    proportions.plot(kind='bar', figsize=(8, 6), width=0.9, color=color_timepoints)
    plt.title('Cell type proportions - cyno - ' + str(celltype))
    plt.xlabel('Cell type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=0)
    plt.ylim([0,1.02])
    plt.tight_layout()
    plt.savefig(os.path.join(base_plots_path,pre + '_' + str(celltype) + '_composition_barplot_cyno_cluster_azimut1_5_scanvi.pdf'),bbox_inches = 'tight')
    plt.show()
    scoda_obj_cyno.obs['Condition'] = scoda_obj_cyno.obs.index 
    scoda_model = mod.CompositionalAnalysis(scoda_obj_cyno, formula="Condition", reference_cell_type="automatic")
    
    sim_results = scoda_model.sample_hmc()
    
    sim_results.summary() 
    
    sim_results.set_fdr(est_fdr=0.25)
    print(sim_results.summary())
    
    print(sim_results.credible_effects())
    
    # saving
    save_path = os.path.join(base_table_path,pre + '_' + str(celltype) + '_scoda_sim_results_cyno.pkl')
    sim_results.save(save_path)

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()